In [25]:
import torch
import functorch as ft
from torch import nn

In [26]:
from sklearn.datasets import make_moons
X, y = make_moons()

In [27]:
batch_size, feature_size = 3, 5

def model(weights,feature_vec):
    # Very simple linear model with activation
    assert feature_vec.dim() == 1
    return feature_vec.dot(weights).relu()

def compute_loss(weights, example, target):
    y = model(weights, example)
    return ((y - target) ** 2).mean()  # MSELoss

weights = torch.randn(feature_size)
examples = torch.randn(batch_size, feature_size)
targets = torch.randn(batch_size)
inputs = (weights,examples, targets)
# grad_weight_per_example = ft.vmap(ft.grad(compute_loss), in_dims=(None, 0, 0))(*inputs)


In [28]:
fn_model = ft.vmap(model, in_dims=(None, *weights.shape))

In [29]:
ft.vmap(model, in_dims=(0, None))(weights[None].repeat(8, 1), examples[0])

tensor([6.4193, 6.4193, 6.4193, 6.4193, 6.4193, 6.4193, 6.4193, 6.4193])

In [31]:
def log_prob(weights):
    return examples[0].dot(weights).relu()

In [33]:
ft.vmap(log_prob, in_dims=(0))(weights[None].repeat(8, 1))

tensor([6.4193, 6.4193, 6.4193, 6.4193, 6.4193, 6.4193, 6.4193, 6.4193])

In [6]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.linear1 = nn.Linear(2, 32)
        self.gelu = nn.GELU()
        self.linear2 = nn.Linear(32, 2)
        
    def forward(self, x):
        h = self.gelu(self.linear1(x))
        return self.linear2(h)
        

In [7]:
model = NN()

In [8]:
fnmodel, params = ft.make_functional(model)

In [9]:
fnmodel(params, torch.randn(2, 2))

tensor([[0.0536, 0.0758],
        [0.2531, 0.3768]], grad_fn=<AddmmBackward0>)

In [10]:
def flatten(params):
    return torch.cat([p.flatten() for p in params])


def unflatten(model, flattened_params):
    if flattened_params.dim() != 1:
        raise ValueError('Expecting a 1d flattened_params')
    params_list = []
    i = 0
    for val in list(model.parameters()):
        length = val.nelement()
        param = flattened_params[i:i+length].view_as(val)
        params_list.append(nn.Parameter(param))
        i += length

    return tuple(params_list)

In [11]:
params_flat = flatten(params)

In [12]:
unflatten(model, params_flat)

(Parameter containing:
 tensor([[-0.4734, -0.0905],
         [-0.5382,  0.1601],
         [-0.5453, -0.4463],
         [-0.5018,  0.3901],
         [-0.6449, -0.5773],
         [-0.0404, -0.1575],
         [-0.4734, -0.5237],
         [-0.0661, -0.1102],
         [-0.4577,  0.4668],
         [ 0.5794, -0.2985],
         [-0.2938, -0.5513],
         [-0.2006,  0.1411],
         [-0.5035, -0.0588],
         [ 0.0116,  0.2855],
         [-0.2573,  0.0981],
         [ 0.3204, -0.5043],
         [-0.1873, -0.0117],
         [ 0.5136, -0.6724],
         [ 0.0550,  0.5823],
         [-0.0532,  0.2993],
         [-0.6140, -0.5306],
         [ 0.0199, -0.5860],
         [ 0.1315,  0.5676],
         [ 0.4479,  0.4941],
         [-0.0053,  0.3060],
         [ 0.6515,  0.1460],
         [ 0.6849,  0.0605],
         [-0.4769, -0.0064],
         [-0.3080,  0.5595],
         [-0.5831, -0.2766],
         [-0.6860,  0.5402],
         [-0.5606,  0.1150]], requires_grad=True),
 Parameter containing:
 ten

In [13]:
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).long()

In [14]:
# y = torch.nn.functional.one_hot(y, 2)

In [15]:
prior = torch.distributions.Normal(torch.zeros(len(params_flat)), 1 * torch.ones(len(params_flat)))




In [16]:
def log_prob(params_flat):
    params = unflatten(model, params_flat)
    logits = fnmodel(params, X)
    
    ce = torch.nn.functional.cross_entropy(logits, y, reduction='none')
    
    nll = -ce.mean()
    
    print(nll)
    
    pl = prior.log_prob(params_flat).sum() / len(X)
    
    return nll + pl
    
    
    
    
    

In [17]:
log_prob(params_flat)

tensor(-0.7054, grad_fn=<NegBackward0>)


tensor(-2.2852, grad_fn=<AddBackward0>)

In [18]:
import hamiltorch

In [19]:

N = 1024
burn = 512
N_nuts = N + burn
step_size = 2e-4
L = 20

samples = hamiltorch.sample(
    log_prob, 
    params_init=params_flat, 
    num_samples=N_nuts, 
    num_steps_per_sample=L, 
    step_size=step_size, 
    burn=burn, 
    desired_accept_rate=0.8,
    sampler=hamiltorch.Sampler.HMC_NUTS,
    debug=True)

Sampling (Sampler.HMC; Integrator.IMPLICIT)
Time spent  | Time remain.| Progress             | Samples   | Samples/sec
tensor(-0.7054, grad_fn=<NegBackward0>)
tensor(-0.7054, grad_fn=<NegBackward0>)
tensor(-0.7055, grad_fn=<NegBackward0>)
tensor(-0.7055, grad_fn=<NegBackward0>)
tensor(-0.7055, grad_fn=<NegBackward0>)
tensor(-0.7055, grad_fn=<NegBackward0>)
tensor(-0.7055, grad_fn=<NegBackward0>)
tensor(-0.7056, grad_fn=<NegBackward0>)
tensor(-0.7056, grad_fn=<NegBackward0>)
tensor(-0.7056, grad_fn=<NegBackward0>)
tensor(-0.7056, grad_fn=<NegBackward0>)
tensor(-0.7057, grad_fn=<NegBackward0>)
tensor(-0.7057, grad_fn=<NegBackward0>)
tensor(-0.7057, grad_fn=<NegBackward0>)
tensor(-0.7057, grad_fn=<NegBackward0>)
tensor(-0.7057, grad_fn=<NegBackward0>)
tensor(-0.7058, grad_fn=<NegBackward0>)
tensor(-0.7058, grad_fn=<NegBackward0>)
tensor(-0.7058, grad_fn=<NegBackward0>)
tensor(-0.7058, grad_fn=<NegBackward0>)
tensor(-0.7058, grad_fn=<NegBackward0>)
tensor(-0.7059, grad_fn=<NegBackward0>)
t

tensor(-0.8611, grad_fn=<NegBackward0>)--------- |   10/1536 | 43.71       
tensor(-0.8611, grad_fn=<NegBackward0>)
tensor(-0.8784, grad_fn=<NegBackward0>)
tensor(-0.8941, grad_fn=<NegBackward0>)
tensor(-0.9078, grad_fn=<NegBackward0>)
tensor(-0.9196, grad_fn=<NegBackward0>)
tensor(-0.9293, grad_fn=<NegBackward0>)
tensor(-0.9369, grad_fn=<NegBackward0>)
tensor(-0.9425, grad_fn=<NegBackward0>)
tensor(-0.9462, grad_fn=<NegBackward0>)
tensor(-0.9480, grad_fn=<NegBackward0>)
tensor(-0.9483, grad_fn=<NegBackward0>)
tensor(-0.9472, grad_fn=<NegBackward0>)
tensor(-0.9451, grad_fn=<NegBackward0>)
tensor(-0.9423, grad_fn=<NegBackward0>)
tensor(-0.9391, grad_fn=<NegBackward0>)
tensor(-0.9359, grad_fn=<NegBackward0>)
tensor(-0.9332, grad_fn=<NegBackward0>)
tensor(-0.9313, grad_fn=<NegBackward0>)
tensor(-0.9305, grad_fn=<NegBackward0>)
tensor(-0.9312, grad_fn=<NegBackward0>)
tensor(-0.9337, grad_fn=<NegBackward0>)
tensor(-0.9337, grad_fn=<NegBackward0>)
Step: 10, Current Hamiltoninian: 111.5393829

tensor(-0.7070, grad_fn=<NegBackward0>)--------- |   21/1536 | 41.69       
tensor(-0.7070, grad_fn=<NegBackward0>)
tensor(-0.7097, grad_fn=<NegBackward0>)
tensor(-0.7162, grad_fn=<NegBackward0>)
tensor(-0.7270, grad_fn=<NegBackward0>)
tensor(-0.7426, grad_fn=<NegBackward0>)
tensor(-0.7632, grad_fn=<NegBackward0>)
tensor(-0.7892, grad_fn=<NegBackward0>)
tensor(-0.8207, grad_fn=<NegBackward0>)
tensor(-0.8577, grad_fn=<NegBackward0>)
tensor(-0.9002, grad_fn=<NegBackward0>)
tensor(-0.9480, grad_fn=<NegBackward0>)
tensor(-1.0011, grad_fn=<NegBackward0>)
tensor(-1.0590, grad_fn=<NegBackward0>)
tensor(-1.1216, grad_fn=<NegBackward0>)
tensor(-1.1885, grad_fn=<NegBackward0>)
tensor(-1.2593, grad_fn=<NegBackward0>)
tensor(-1.3338, grad_fn=<NegBackward0>)
tensor(-1.4117, grad_fn=<NegBackward0>)
tensor(-1.4924, grad_fn=<NegBackward0>)
tensor(-1.5758, grad_fn=<NegBackward0>)
tensor(-1.6615, grad_fn=<NegBackward0>)
tensor(-1.6615, grad_fn=<NegBackward0>)
Step: 21, Current Hamiltoninian: 90.32969665

tensor(-2.6210, grad_fn=<NegBackward0>)
tensor(-2.6315, grad_fn=<NegBackward0>)
tensor(-2.6422, grad_fn=<NegBackward0>)
tensor(-2.6530, grad_fn=<NegBackward0>)
tensor(-2.6639, grad_fn=<NegBackward0>)
tensor(-2.6749, grad_fn=<NegBackward0>)
tensor(-2.6860, grad_fn=<NegBackward0>)
tensor(-2.6972, grad_fn=<NegBackward0>)
tensor(-2.7085, grad_fn=<NegBackward0>)
tensor(-2.7199, grad_fn=<NegBackward0>)
tensor(-2.7314, grad_fn=<NegBackward0>)
tensor(-2.7431, grad_fn=<NegBackward0>)
tensor(-2.7548, grad_fn=<NegBackward0>)
tensor(-2.7666, grad_fn=<NegBackward0>)
tensor(-2.7786, grad_fn=<NegBackward0>)
tensor(-2.7906, grad_fn=<NegBackward0>)
tensor(-2.8027, grad_fn=<NegBackward0>)
tensor(-2.8150, grad_fn=<NegBackward0>)
tensor(-2.8273, grad_fn=<NegBackward0>)
tensor(-2.8398, grad_fn=<NegBackward0>)
tensor(-2.8398, grad_fn=<NegBackward0>)
Step: 30, Current Hamiltoninian: 71.70816802978516, Proposed Hamiltoninian: 71.93740844726562
Accept rho: -0.22924041748046875
tensor(-2.8398, grad_fn=<NegBackw

tensor(-2.0189, grad_fn=<NegBackward0>)
tensor(-1.9734, grad_fn=<NegBackward0>)
tensor(-1.9288, grad_fn=<NegBackward0>)
tensor(-1.8854, grad_fn=<NegBackward0>)
tensor(-1.8431, grad_fn=<NegBackward0>)
tensor(-1.8021, grad_fn=<NegBackward0>)
tensor(-1.7627, grad_fn=<NegBackward0>)
tensor(-1.7250, grad_fn=<NegBackward0>)
tensor(-1.6893, grad_fn=<NegBackward0>)
tensor(-1.6893, grad_fn=<NegBackward0>)
Step: 38, Current Hamiltoninian: 79.05007934570312, Proposed Hamiltoninian: 78.12238311767578
Accept rho: 0.0
tensor(-1.6893, grad_fn=<NegBackward0>)
tensor(-1.6893, grad_fn=<NegBackward0>)
tensor(-1.5905, grad_fn=<NegBackward0>)
tensor(-1.4942, grad_fn=<NegBackward0>)
tensor(-1.4007, grad_fn=<NegBackward0>)
tensor(-1.3103, grad_fn=<NegBackward0>)
tensor(-1.2233, grad_fn=<NegBackward0>)
tensor(-1.1400, grad_fn=<NegBackward0>)
tensor(-1.0608, grad_fn=<NegBackward0>)
tensor(-0.9861, grad_fn=<NegBackward0>)
tensor(-0.9163, grad_fn=<NegBackward0>)
tensor(-0.8518, grad_fn=<NegBackward0>)
tensor(-0.

...

tensor(-1.5726, grad_fn=<NegBackward0>)
Step: 1497, Current Hamiltoninian: 99.68508911132812, Proposed Hamiltoninian: 99.90890502929688
Accept rho: -0.22381591796875
tensor(-1.5726, grad_fn=<NegBackward0>)######### | 1498/1536 | 42.13       
tensor(-1.5726, grad_fn=<NegBackward0>)
tensor(-1.5638, grad_fn=<NegBackward0>)
tensor(-1.5557, grad_fn=<NegBackward0>)
tensor(-1.5484, grad_fn=<NegBackward0>)
tensor(-1.5418, grad_fn=<NegBackward0>)
tensor(-1.5358, grad_fn=<NegBackward0>)
tensor(-1.5305, grad_fn=<NegBackward0>)
tensor(-1.5261, grad_fn=<NegBackward0>)
tensor(-1.5226, grad_fn=<NegBackward0>)
tensor(-1.5200, grad_fn=<NegBackward0>)
tensor(-1.5183, grad_fn=<NegBackward0>)
tensor(-1.5173, grad_fn=<NegBackward0>)
tensor(-1.5170, grad_fn=<NegBackward0>)
tensor(-1.5174, grad_fn=<NegBackward0>)
tensor(-1.5185, grad_fn=<NegBackward0>)
tensor(-1.5204, grad_fn=<NegBackward0>)
tensor(-1.5232, grad_fn=<NegBackward0>)
tensor(-1.5271, grad_fn=<NegBackward0>)
tensor(-1.5320, grad_fn=<NegBackward0>

tensor(-2.2857, grad_fn=<NegBackward0>)
tensor(-2.3206, grad_fn=<NegBackward0>)
tensor(-2.3560, grad_fn=<NegBackward0>)
tensor(-2.3918, grad_fn=<NegBackward0>)
tensor(-2.4281, grad_fn=<NegBackward0>)
tensor(-2.4649, grad_fn=<NegBackward0>)
tensor(-2.5022, grad_fn=<NegBackward0>)
tensor(-2.5400, grad_fn=<NegBackward0>)
tensor(-2.5400, grad_fn=<NegBackward0>)
Step: 1506, Current Hamiltoninian: 109.7657241821289, Proposed Hamiltoninian: 110.43328094482422
Accept rho: -0.6675567626953125
tensor(-2.5400, grad_fn=<NegBackward0>)
tensor(-2.5400, grad_fn=<NegBackward0>)
tensor(-2.5390, grad_fn=<NegBackward0>)
tensor(-2.5391, grad_fn=<NegBackward0>)
tensor(-2.5402, grad_fn=<NegBackward0>)
tensor(-2.5421, grad_fn=<NegBackward0>)
tensor(-2.5448, grad_fn=<NegBackward0>)
tensor(-2.5484, grad_fn=<NegBackward0>)
tensor(-2.5531, grad_fn=<NegBackward0>)
tensor(-2.5590, grad_fn=<NegBackward0>)
tensor(-2.5660, grad_fn=<NegBackward0>)
tensor(-2.5742, grad_fn=<NegBackward0>)
tensor(-2.5833, grad_fn=<NegBac

tensor(-4.3227, grad_fn=<NegBackward0>)
tensor(-4.3939, grad_fn=<NegBackward0>)
tensor(-4.4657, grad_fn=<NegBackward0>)
tensor(-4.5381, grad_fn=<NegBackward0>)
tensor(-4.6111, grad_fn=<NegBackward0>)
tensor(-4.6847, grad_fn=<NegBackward0>)
tensor(-4.7589, grad_fn=<NegBackward0>)
tensor(-4.8337, grad_fn=<NegBackward0>)
tensor(-4.9091, grad_fn=<NegBackward0>)
tensor(-4.9850, grad_fn=<NegBackward0>)
tensor(-5.0616, grad_fn=<NegBackward0>)
tensor(-5.1386, grad_fn=<NegBackward0>)
tensor(-5.2163, grad_fn=<NegBackward0>)
tensor(-5.2944, grad_fn=<NegBackward0>)
tensor(-5.3731, grad_fn=<NegBackward0>)
tensor(-5.4524, grad_fn=<NegBackward0>)
tensor(-5.5321, grad_fn=<NegBackward0>)
tensor(-5.6124, grad_fn=<NegBackward0>)
tensor(-5.6931, grad_fn=<NegBackward0>)
tensor(-5.7744, grad_fn=<NegBackward0>)
tensor(-5.8561, grad_fn=<NegBackward0>)
tensor(-5.8561, grad_fn=<NegBackward0>)
Step: 1515, Current Hamiltoninian: 102.74591064453125, Proposed Hamiltoninian: 104.27934265136719
REJECT
tensor(-4.3227,

tensor(-6.1718, grad_fn=<NegBackward0>)
tensor(-6.1402, grad_fn=<NegBackward0>)
tensor(-6.1086, grad_fn=<NegBackward0>)
tensor(-6.0770, grad_fn=<NegBackward0>)
tensor(-6.0454, grad_fn=<NegBackward0>)
tensor(-6.0138, grad_fn=<NegBackward0>)
tensor(-5.9821, grad_fn=<NegBackward0>)
tensor(-5.9504, grad_fn=<NegBackward0>)
tensor(-5.9187, grad_fn=<NegBackward0>)
tensor(-5.8871, grad_fn=<NegBackward0>)
tensor(-5.8554, grad_fn=<NegBackward0>)
tensor(-5.8237, grad_fn=<NegBackward0>)
tensor(-5.7921, grad_fn=<NegBackward0>)
tensor(-5.7605, grad_fn=<NegBackward0>)
tensor(-5.7289, grad_fn=<NegBackward0>)
tensor(-5.6974, grad_fn=<NegBackward0>)
tensor(-5.6659, grad_fn=<NegBackward0>)
tensor(-5.6344, grad_fn=<NegBackward0>)
tensor(-5.6030, grad_fn=<NegBackward0>)
tensor(-5.6030, grad_fn=<NegBackward0>)
Step: 1523, Current Hamiltoninian: 87.12815856933594, Proposed Hamiltoninian: 86.49606323242188
Accept rho: 0.0
tensor(-5.6030, grad_fn=<NegBackward0>)
tensor(-5.6030, grad_fn=<NegBackward0>)
tensor(-

tensor(-3.4032, grad_fn=<NegBackward0>)
tensor(-3.4116, grad_fn=<NegBackward0>)
tensor(-3.4197, grad_fn=<NegBackward0>)
tensor(-3.4274, grad_fn=<NegBackward0>)
tensor(-3.4348, grad_fn=<NegBackward0>)
tensor(-3.4419, grad_fn=<NegBackward0>)
tensor(-3.4486, grad_fn=<NegBackward0>)
tensor(-3.4549, grad_fn=<NegBackward0>)
tensor(-3.4549, grad_fn=<NegBackward0>)
Step: 1531, Current Hamiltoninian: 87.56952667236328, Proposed Hamiltoninian: 87.75968170166016
Accept rho: -0.190155029296875
tensor(-3.4549, grad_fn=<NegBackward0>)
tensor(-3.4549, grad_fn=<NegBackward0>)
tensor(-3.6007, grad_fn=<NegBackward0>)
tensor(-3.7483, grad_fn=<NegBackward0>)
tensor(-3.8979, grad_fn=<NegBackward0>)
tensor(-4.0492, grad_fn=<NegBackward0>)
tensor(-4.2023, grad_fn=<NegBackward0>)
tensor(-4.3570, grad_fn=<NegBackward0>)
tensor(-4.5132, grad_fn=<NegBackward0>)
tensor(-4.6709, grad_fn=<NegBackward0>)
tensor(-4.8300, grad_fn=<NegBackward0>)
tensor(-4.9904, grad_fn=<NegBackward0>)
tensor(-5.1522, grad_fn=<NegBackw

In [20]:
samples = torch.stack(samples)

In [45]:
logprob_vmap = ft.vmap(log_prob)

In [46]:
logprob_vmap(samples)

RuntimeError: size mismatch, got 1024, 1024x162,5

In [21]:
y_ = fnmodel(unflatten(model, samples[-1]), X)

In [22]:
y_hat = y_.argmax(-1)

In [23]:
torch.mean((y_hat == y).float())

tensor(0.8400)

In [35]:
models = [NN(), NN()]

In [36]:
fmodel, params, buffers = ft.combine_state_for_ensemble(models)


In [40]:
params[0].shape

torch.Size([2, 32, 2])

In [41]:
params[1].shape

torch.Size([2, 32])

In [43]:
params[2].shape

torch.Size([2, 2, 32])

In [44]:
params[3].shape

torch.Size([2, 2])